## Todo:
Open in google colab -> [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg#left)](https://colab.research.google.com/github/SuperChange001/deeplearning_labs/blob/main/Lab01.ipynb)

- [ ] Google Colab introduction
- [ ] AI programming 101
- [ ] setup the environment(e.g, install/import all packages that we need)
- [ ] create your fist model
- [ ] data preparation, prepare train/test set, visualize data
- [ ] Train model, test model
- [ ] Adding useful log info during training, and visualization for understanding the trained model(e.g confusion matrix)
- [ ] tensor board(maybe)
- [ ] Store/load the trained model

In [ ]:
sss